In [6]:
import time
from time import sleep
import bs4 as bs
import urllib.request
from urllib.error import HTTPError
import pandas as pd
import re
from tqdm import tnrange, tqdm_notebook

def cleanhtml(raw_html): #membersihkan paragraf dari tag html
  cleanr = re.compile('<.*?>') #set tag yang akan di hapus
  cleantext = re.sub(cleanr, '', raw_html) #hapus tag htmlnya
  return cleantext

def crawl_1para(soup, raw_para): #menarik paragraf dalam 1 halaman berita
    for para in soup.find_all('p'): #mencari paragraf dengan menggunakan tag html p
        flag1 = "" #inisiasi flag 1
        flag2 = "" #insiasi flag 2
        flag1 = str((para.get('class'))) #mendapatkan class dari teks paragraf
        try:
            flag2 = (para.find('a').get('title')) #mendapatkan title dari teks paragraf
        except:
            flag2 = "" #jika tidak ada di abaikan dengan menggunakan string kosong
            pass
        flag3 = str(para).lower().find('selanjutnya') #mencari kata selanjutnya
        flag4 = str(para).lower().find('selengkapnya') #mencari kata selengkapnya
        flag5 = str(para).lower().find('berikutnya') #mencari kata berikutnya
        
        if(flag1=="['baca']" or flag2=="halflink" or flag3!=-1 or flag4!=-1 or flag5!=-1):
            pass #jika ditemukan kata tersebut berita yang diambil adalah iklan
        else: #selain itu paragraf akan di simpan di dalam raw_para
            para = para.text 
            raw_para = raw_para + para
    return raw_para

def crawl_headlines(soup): #menarik headlines semua berita di dalam 1 halaman index
    headlines = soup.find_all('a', href=True, title=True, class_="fbo f18 ln24") #mencari headlines
    return headlines

def crawl_times(soup, times): #menarik data waktu
    try:
        times.append(soup.find('time').text) #memasukan data waktu berita ke dalam list
    except:
        times.append('') #jika data tidak ditemukan memasukan string kosong ke dalam list
    return times

def crawl_lokasi(soup, locations): #menarik data lokasi berita
    try: 
        locations.append(soup.find('strong').text) #memasukan data lokasi berita ke dalam list
    except:
        locations.append('') #jika data tidak ditemukan memasukan string kosong ke dalam list
    return locations

def crawl_keyword(soup, keywords): #menarik data tag berita
    keywords.append(soup.find(attrs={'name':"news_keywords"}).get('content')) #memasukan tag berita ke dalam list
    return keywords

def crawl_news_para(soup, paragraphs): #mengambil data paragraf 1 berita
    raw_para = "" #inisiasi data paragraf
    raw_para = crawl_1para(soup, raw_para) #mengambil data paragraf di dalam 1 halaman
    while True: #pencarian nama paragraf dilakukan hingga halaman terakhir paragraf
        try:
            url = soup.find('a', class_=("bgblue white ptb5 plr10 f18")).get('href') #mencari nama tombol url
            txt = soup.find('a', class_=("bgblue white ptb5 plr10 f18")).text #mencari nama tombolnya
            if(txt == 'Halaman selanjutnya '): #jika masih ada halaman selanjutnya
                try:
                    sauce = urllib.request.urlopen(url) #mengakses url
                    soup = bs.BeautifulSoup(sauce,'lxml') #membaca html
                    raw_para = crawl_1para(soup, raw_para) #Mengambil data paragraf di dalam 1 halaman
                except(HTTPError, ValueError): #Terjadi koneksi error mengulangi akses url
                    pass
            else: #Halaman terakhir berita
                break
        except: #Berita hanya 1 halaman, pencarian paragraf dihentikan
            break
    clean_para = cleanhtml(raw_para) #membersihkan data paragraf
    paragraphs.append(clean_para) #memasukan paragraf yang sudah bersih ke dalam list
    return paragraphs

def crawl_editor(soup, editors): #menarik data editor
    try:
        raw_editor = soup.find('div', attrs={'id':'editor'}).string #mencari editor di dalam url
        editors.append(raw_editor[11:-2]) #memasukan editor ke dalam list
    except: #jika ditemukan editor dimasukan string kosong
        editors.append('')
    return editors

def outside_crawl(tags, labels, date_start, date_end):
    #Inisiasi
    titles = [] #judul berita/headlines
    links = [] #link berita
    dates = [] #tanggal berita
    resumes = [] #resume berita
    label = [] #label berita
    tag = [] #tag yang digunakan dalam pencarian
    
    for i in tqdm_notebook(range(len(tags)), desc='Outside Crawler Progress'): #Loading bar, Perulangan sebanyak jumlah tag
        url = 'http://www.tribunnews.com/tag/' + tags[i] #set url tag
        sauce = urllib.request.urlopen(url) #mengakses url
        soup = bs.BeautifulSoup(sauce,'lxml') #membaca html url

        while True: #selama bukan halaman akhir dan tanggal masih dalam pencarian
            try:
                url = soup.find("a", text=("Next")).get('href') #mencari link yang mengarahkan ke halaman berikutnya
                try:
                    sauce = urllib.request.urlopen(url) #mengakses halaman berikutnya
                    soup = bs.BeautifulSoup(sauce,'lxml') #membaca halaman berikutnya
                except (HTTPError, ValueError): #jika terjadi koneksi error
                    print('error!') #print error dan mengulang try
                    pass

                headlines = crawl_headlines(soup) #crawl semua headlines dalam 1 halaman index
                for x in headlines: #menunjuk headlines 1 persatu
                    link = x.get('href') #mencari link berita di dalam headlines
                    
                    awal = link.find('/', 28) #mencari tanggal berita di dalam url
                    date_now = link[awal+1:awal+11] #mencari tanggal berita di dalam url

                    if(date_now > date_end): #Cek jika tanggal berita sekarang lebih baru dibandingkan tanggal akhir
                        continue #Berita tidak akan dimasukan ke dalam list

                    if(date_now < date_start): #Cek jika tanggal berita sekarang lebih lama dibandingkan tanggal awal
                        break #Pencarian dihentikan
                    
                    titles.append(x.get('title')) #memasukan judul tiap berita ke dalam list
                    links.append(link) #memasukan link tiap berita ke dalam list
                    label.append(labels[i]) #memasukan label tiap berita ke dalam list
                    tag.append(tags[i]) #memasukan tag yang digunakan dalam pencarian ke dalam list

            #Jika halaman terakhir
            except AttributeError:
                break
    return titles, links, label, tag

def inside_crawl(links):
    #Insiasi
    times = [] #waktu
    locations = [] #lokasi
    keywords = [] #3-4 tags berita
    paragraphs = [] #paragraf/isi berita
    editors = [] #editor
    
    for i in tqdm_notebook(range(len(links)), desc='Inside Crawl Progress'): #loading bar, menarik data dari setiap link
        url = links[i] #menunjuk link 1 persatu
        sauce = urllib.request.urlopen(url) #mengakses html link
        soup = bs.BeautifulSoup(sauce,'lxml') #membaca html link
        
        #Crawling
        times = crawl_times(soup, times) #waktu
        locations = crawl_lokasi(soup, locations) #lokasi
        keywords = crawl_keyword(soup, keywords) #3-4 tags berita
        paragraphs = crawl_news_para(soup, paragraphs) #paragraf/isi  berita
        editors = crawl_editor(soup, editors) #editor

    return times, locations, keywords, paragraphs, editors

In [7]:
tags = ['joko-widodo', 'prabowo-subianto'] #list tags yang mau di crawl
nama_label = ['jokowi', 'prabowo'] #nama label dari setiap tag
tanggal_mulai = '2018/08/01' #tanggal awal pencarian
tanggal_akhir = '2018/10/29' #tanggal akhir pencarian

titles, links, label, tag = outside_crawl(tags, nama_label, tanggal_mulai, tanggal_akhir) #crawl bagian luar berita (mengakses index)
times, locations, keywords, paragraphs, editors = inside_crawl(links) #crawl isi berita (sudah mengakses link berita)

tribun = {'headline' :titles, 'dates' :times, 'location' :locations, 'link' :links,
          'paragraph' :paragraphs, 'label' :label, 'tag' :tag, 'keywords' :keywords, 'editor' :editors} #Data Framing step 1

df_tribun = pd.DataFrame(tribun, columns = tribun.keys()) #Data Framing step 2
df_tribun.to_csv('output.csv')
df_tribun.head()

,headline,dates,location,link,paragraph,label,tag,keywords,editor
0,Silaturahmi dengan Pimpinan Ponpes se-Malang R...,"Senin, 29 Oktober 2018 21:37 WIB","TRIBUNNEWS.COM, MALANG -",http://www.tribunnews.com/regional/2018/10/29/...,"TRIBUNNEWS.COM, MALANG - Calon wakil presiden ...",jokowi,joko-widodo,"Pilpres 2019, Maruf Amin, Malang, cawapres, Jo...",Sugiyarto
1,Indonesia Memiliki Program Pengurangan Polusi ...,"Senin, 29 Oktober 2018 20:21 WIB","TRIBUNNEWS.COM, BADUNG",http://www.tribunnews.com/nasional/2018/10/29/...,"TRIBUNNEWS.COM, BADUNG - Presiden Joko Widodo ...",jokowi,joko-widodo,"Joko Widodo, Our Ocean Conference 2018, Nusa D...",Toni Bramantoro
2,Pemerintah Belum Pikirkan Soal Sanksi dan Bata...,"Senin, 29 Oktober 2018 20:18 WIB","Laporan Wartawan Tribunnews.com, Seno Tri Suli...",http://www.tribunnews.com/bisnis/2018/10/29/pe...,"Laporan Wartawan Tribunnews.com, Seno Tri Suli...",jokowi,joko-widodo,"Pesawat Lion Air Jatuh, Lion Air, Joko Widodo,...",Choirul Arifin
3,Jokowi: Kita Lakukan Upaya Terbaik Menemukan d...,"Senin, 29 Oktober 2018 15:12 WIB","TRIBUNNEWS.COM, JAKARTA",http://www.tribunnews.com/nasional/2018/10/29/...,"TRIBUNNEWS.COM, JAKARTA - Presiden Joko Widodo...",jokowi,joko-widodo,"Pesawat Lion Air Jatuh, Joko Widodo, Basarnas,...",Johnson Simanjuntak
4,TKN Jokowi-Ma'ruf: Salam Satu Jempol Kobarkan ...,"Senin, 29 Oktober 2018 08:31 WIB","Laporan Wartawan Tribunnews.com, Dennis Destry...",http://www.tribunnews.com/nasional/2018/10/29/...,"Laporan Wartawan Tribunnews.com, Dennis Destry...",jokowi,joko-widodo,"Pilpres 2019, Hasto Kristiyanto, Joko Widodo, ...",Hendra Gunawan
